In [1]:
col_names = [
    'dv-fv',
    'dv', 
    'fv'
]

In [6]:
import configparser
import warnings
import pandas as pd
from pymongo import MongoClient
import json
import requests
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore", category=DeprecationWarning)

config = configparser.ConfigParser()
config.read('config.ini')
host = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

client = MongoClient(host, int(port))
db = client['Media-cloud']

In [3]:
def get_text(url):
    text = ''
    try:
        if '/?nk=' in url:
            return ''
        
        html = requests.get(url).text
        soup = BeautifulSoup(html)

        for p in soup.find_all('p'):
            if p.parent.get('class') == ['section']:
                continue
            if p.get('class') == ['_2g0-m']:
                continue
            sentence = p.get_text().strip()
            if (not sentence.startswith('A NOTE') and
                not sentence.startswith('If you have inside') and
                not sentence.startswith('ABC') and
                not sentence.startswith('Read') and
                not sentence.startswith('Sign up') and
                not sentence.startswith('Stay Informed') and
                not sentence.startswith('For independent news') and
                not sentence.startswith("Don't miss") and 
                not sentence.startswith('Follow') and
                '<!' not in sentence
               ) and (
                sentence.endswith('.') or 
                sentence.endswith("'") or
                sentence.endswith('"') or
                sentence.endswith("?") or
                sentence.endswith("!") or
                sentence.endswith(";") or
                sentence.endswith(",") or
                sentence.endswith(")") or
                sentence.endswith(":")
            ):
                text += sentence + ' '
                
        return text.strip()
    except:
        return text.strip()

In [ ]:
for col_name in col_names:
    counter = 0
    print('Now processing:', col_name+'.csv')
    
    df = pd.read_csv('input/'+col_name+'.csv')
    df['text'] = ''
    for i, row in df.iterrows():
        url = row['url']
        df.at[i, 'text'] = get_text(url)
        
        counter += 1
        if counter % 100 == 0:
            print(counter, 'records processed')
            
    records = json.loads(df.to_json(orient='records'))
    
    print(col_name+'.csv finished\n--------------------\n')

In [ ]:
df = pd.read_csv('input/dv.csv')
col = db['dv']
n = 0

for _, row in df.iterrows()[-200]:
    story_id = row['stories_id']
    if col.find({'stories_id': story_id}).count() == 0:
        url = row['url']
        if url.endswith('.mp3'):
            continue
        text = get_text(url)
        record = {
            "stories_id" : row['stories_id'],
            "publish_date" : row['publish_date'],
            "title" : row['title'],
            "url" : row['url'],
            "language" : row['language'],
            "ap_syndicated" : row['ap_syndicated'],
            "themes" : row['themes'],
            "media_id" : row['media_id'],
            "media_name" : row['media_name'],
            "media_url" : row['media_url'],
            "media_pub_country" : row['media_pub_country'],
            "media_pub_state" : row['media_pub_state'],
            "media_language" : row['media_language'],
            "media_about_country" : row['media_about_country'],
            "media_media_type" : row['media_media_type'],
            "text" : text
        }
        col.insert_one(record)
        
    n += 1
    if n % 100 == 0:
        print(n)

In [9]:
print(url)

http://mpegmedia.abc.net.au/ra/MOD/RA_Podcast/PAB_PM_20170313.mp3
